* "Most content" come from CSDN博主「usouvenir」的原創文章，遵循CC 4.0 BY-SA版權協議，轉載請附上原文出處鏈接及本聲明。
* 原文鏈接：https://blog.csdn.net/usouvenir/article/details/107013732

In [47]:
import pandas as pd

In [48]:
tips=pd.read_csv("tips.csv")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# GroupBy
## df.groupby([分組列]).agg({統計列:統計函數,統計列:統計函數})
## df.groupby([分組列])[統計列].mean()

In [64]:
tips.groupby(["day","smoker"]).mean()

total_bill       tip      size
day  smoker                                
Fri  No       18.420000  2.812500  2.250000
     Yes      16.813333  2.714000  2.066667
Sat  No       19.661778  3.102889  2.555556
     Yes      21.276667  2.875476  2.476190
Sun  No       20.506667  3.167895  2.929825
     Yes      24.120000  3.516842  2.578947
Thur No       17.113111  2.673778  2.488889
     Yes      19.190588  3.030000  2.352941

# Pivot Table
## df.pivot_table(index=[ ],columns=[ ],values=[ ],aggfunc=[ ])

In [50]:
tips.pivot_table(index=["day","smoker"])

size       tip  total_bill
day  smoker                                
Fri  No      2.250000  2.812500   18.420000
     Yes     2.066667  2.714000   16.813333
Sat  No      2.555556  3.102889   19.661778
     Yes     2.476190  2.875476   21.276667
Sun  No      2.929825  3.167895   20.506667
     Yes     2.578947  3.516842   24.120000
Thur No      2.488889  2.673778   17.113111
     Yes     2.352941  3.030000   19.190588

### 只聚合tip_pct和size列，根據day進行分組；將smoker放到列上，day放到行上。

In [51]:
tips.groupby(["day","smoker"])[["size","total_bill"]].agg("mean")
# unstacked()

size  total_bill
day  smoker                      
Fri  No      2.250000   18.420000
     Yes     2.066667   16.813333
Sat  No      2.555556   19.661778
     Yes     2.476190   21.276667
Sun  No      2.929825   20.506667
     Yes     2.578947   24.120000
Thur No      2.488889   17.113111
     Yes     2.352941   19.190588

In [52]:
tips.pivot_table(["size","total_bill"],index="day",columns="smoker")

size           total_bill           
smoker        No       Yes         No        Yes
day                                             
Fri     2.250000  2.066667  18.420000  16.813333
Sat     2.555556  2.476190  19.661778  21.276667
Sun     2.929825  2.578947  20.506667  24.120000
Thur    2.488889  2.352941  17.113111  19.190588

---參數解釋---
* data：要應用透視表的數據框；
* values：待聚合的列的名稱，默認聚合所有數值列；
* index：用於分組的列名或其他分組鍵，出現在結果透視表的行；
* columns：用於分組的列名或其他分組鍵，出現在結果透視表的列；
* aggfunc：聚合函數或函數列表，默認為 'mean'，可以是任何對 groupby 有效的函數；
* fill_value：用於替換結果表中的缺失值；
* margins：添加行/列小計和總計，默認為 False；dropna：如果為True，不添加條目都為NA的列；
* margins_name：當margins為True時，行/列小計和總計的名稱；
* observed：僅當是Categoricals時才適用，如果為True：僅顯示類別分組的觀察值，False：顯示類別分組的所有值。

# CrossTab (計算分組頻率)
## pd.crosstab(index=df[ ],columns=df[ ])

* rownames/colnames：行/列；
* normalize：是否標準化。

In [53]:
pd.crosstab(tips["sex"],tips["smoker"])

smoker,No,Yes
sex,,
Female,54,33
Male,97,60


----

In [54]:
df=pd.read_csv("employee.csv")
df.head()

,title,dept,salary,race,gender,hire_date
0,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Male,2015-02-03
1,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Male,1982-02-08
2,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black,Male,1984-11-26
3,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian,Male,2012-03-26
4,CARPENTER,Houston Airport System (HAS),42390.0,White,Male,2013-11-04


In [55]:
df.isnull().sum()

title          0
dept           0
salary       102
race          20
gender         0
hire_date      0
dtype: int64

In [56]:
df["salary"].fillna(value=(df["salary"].mean()))
df["race"].fillna(method="ffill")

0          White
1          White
2          Black
3          Asian
4          White
          ...   
1648       White
1649       Black
1650       White
1651       Asian
1652    Hispanic
Name: race, Length: 1653, dtype: object

In [57]:
pd.crosstab(index=df["race"],columns=df["dept"])

dept,Health & Human Services,Houston Airport System (HAS),Houston Fire Department (HFD),Houston Police Department-HPD,Parks & Recreation,Public Works & Engineering-PWE
race,,,,,,
Asian,11,6,1,39,2,29
Black,53,40,69,171,38,171
Hispanic,27,30,99,151,20,68
Native American,3,1,1,3,0,0
White,12,26,209,269,9,75


In [58]:
df.pivot_table(index=["dept"],columns=["race","gender"],values=["salary"],aggfunc=["mean"])

mean                              \
                                      salary                               
race                                   Asian                       Black   
gender                                Female          Male        Female   
dept                                                                       
Health & Human Services         56350.166667  54167.200000  52989.048780   
Houston Airport System (HAS)    29141.000000  52323.250000  54195.928571   
Houston Fire Department (HFD)            NaN  43528.000000  56413.714286   
Houston Police Department-HPD   55804.333333  60254.242424  49561.808219   
Parks & Recreation                       NaN  48651.000000  36526.666667   
Public Works & Engineering-PWE  79566.750000  64891.440000  43018.133333   

                                                                          \
                                                                           
race                                              Hispanic                 
gender                                  Male        Female          Male   
dept                                                                       
Health & Human Services         67861.000000  35228.000000  40605.600000   
Houston Airport System (HAS)    50407.461538  46254.750000  49888.722222   
Houston Fire Department (HFD)   58126.377049  49146.250000  57580.566667   
Houston Police Department-HPD   60876.886364  47654.000000  61418.972222   
Parks & Recreation              37032.562500  51882.600000  36512.454545   
Public Works & Engineering-PWE  42197.118182  43094.947368  46436.510204   

                                                                            \
                                                                             
race                           Native American                       White   
gender                                  Female          Male        Female   
dept                                                                         
Health & Human Services                54117.0           NaN  58721.833333   
Houston Airport System (HAS)           68299.0           NaN  73362.166667   
Houston Fire Department (HFD)              NaN  78355.000000  53641.333333   
Houston Police Department-HPD              NaN  65682.333333  63782.419355   
Parks & Recreation                         NaN           NaN           NaN   
Public Works & Engineering-PWE             NaN           NaN  73491.782609   

                                              
                                              
race                                          
gender                                  Male  
dept                                          
Health & Human Services         64627.666667  
Houston Airport System (HAS)    63128.500000  
Houston Fire Department (HFD)   62438.764398  
Houston Police Department-HPD   65557.752577  
Parks & Recreation              43630.500000  
Public Works & Engineering-PWE  62216.352941

In [73]:
df.groupby(["race","gender"]).agg({"salary":"mean"})

salary
race            gender              
Asian           Female  58304.222222
                Male    60622.956522
Black           Female  48133.381643
                Male    51853.000000
Hispanic        Female  44216.960000
                Male    55493.064057
Native American Female  58844.333333
                Male    68850.500000
White           Female  66415.527778
                Male    63439.195745